# JP Morgan | Interest Rate Mathematics


In [231]:
from Binomial_Fixed import ratecurves
from Binomial_Fixed import treasury_cmds
import pandas as pd
import numpy as np
import datetime as dt
from scipy.optimize import fsolve

In [232]:


# Initialize the paramaters of the loan
loan_amount = 18000
apr = 5.29
loan_term = 60

# Get a monthly percentage rate
apr /= 100
mpr = apr / 12

# Calculate the Monthly Payment for a loan
monthly_payment = (loan_amount * mpr)/(1-(1+mpr) ** -loan_term)

def make_payment(principal, mpr, monthly_payment):
    '''Makes a 'payment' by subtracting and updated payment amount from the 
    principal. Returns the principal remaining, and the amount of principal and interest paid
    '''

    current_interest_payment = principal * mpr
    current_principal_payment = monthly_payment - current_interest_payment
    
    principal -= current_principal_payment
    
    return [round(principal, 2), round(current_principal_payment, 2), round(current_interest_payment, 2)] 

def main(principal, term_remaining, monthly_payment, mpr):
    '''Returns an Amortization Table in the form of a DataFrame
    '''
    
    payments = [[principal, 0, 0, 0]]
    total_interest = 0
    
    while principal > 0 and term_remaining > 0:
        payment = make_payment(principal, mpr, monthly_payment)
        principal = payment[0]
        term_remaining -= 1
        total_interest += payment[2]
        payment.append(total_interest)
        payments.append(payment)

    amortization_table = pd.DataFrame(data=payments,
                                      columns=['Principal Remaining',
                                              'Current Principal Payment',
                                              'Current Interest Payment',
                                              'Total Interest Paid'])
    return amortization_table

In [233]:
schedule = main(loan_amount, loan_term, monthly_payment,mpr)

In [234]:
def compound_to_equivalent(rate:float, initial_compound_freq: int,compound_freq:int):
    """
    rate: enter interest rate (Example: 4.23)
    initial_compound_freq: number of periods interest is paid
    compound_freq: frequency you want interest to be compounded (set equal to 1 to get effective annual rate)
    returns equivalent interest rate 

    """
    rate=  rate*1e-2
    Rc = initial_compound_freq*np.log(1+(rate/initial_compound_freq))
    annual_rate = compound_freq*(np.exp(Rc/compound_freq)-1)
    #annual_rate = (1+(rate/compound))**(compound) -1
    return f"Equivalent Annual Interest Rate is {annual_rate :.2%}."
def compound_to_continuous(rate:float,compound_freq:int):
    """
    rate: enter interest rate(Example 3.202)
    compound_freq: enter interval that represents number of times interest is compounded per annum
    returns equivalent continously compounded rate
    """
    rate = rate*1e-2
    Rc = compound_freq*np.log(1+(rate/compound_freq))
    return f"Continuously compounded equivalent rate is {Rc :.2%}."

def calculate_interest_accrued(rate:float,Notional:float,compound:int,coup_0:str,date_0:str,next_coup_date:str):

    """
    rate: enter interest rate(Example 3.202)
    Notional: Notional value of investment
    compound_freq : enter number of times interest is paid
    coup_0: last coupon date (Example: '2005-08-22')
    date_0: current date 
    next_coup_date: next coupon date
    returns interest accrued during time period
    """
    rate = rate*1e-2
    N = Notional
    coup_0 = dt.datetime.strptime(coup_0,"%Y-%m-%d")
    date_0 = dt.datetime.strptime(date_0,"%Y-%m-%d")
    next_coup_date = dt.datetime.strptime(next_coup_date,"%Y-%m-%d")
    ref_period = (next_coup_date-coup_0).days
    actual_days = (date_0-coup_0).days
    frac_days = actual_days/360
    accrued = (N)*(rate*frac_days)
    return np.round(accrued,2)


    
    




In [235]:
compound_to_equivalent(5,2,12)

'Equivalent Annual Interest Rate is 4.95%.'

In [236]:
calculate_interest_accrued(rate = 6.25,Notional=100000,compound = 2,coup_0='2023-03-15',date_0='2023-09-15',next_coup_date='2023-09-15')

3194.44

In [237]:
calculate_interest_accrued(8,100,2,'2023-03-01',date_0='2023-07-03',next_coup_date='2023-09-01')

2.76

In [238]:
compound_to_continuous(5,2)

'Continuously compounded equivalent rate is 4.94%.'

In [240]:
schedule['Current Principal Payment'].sum()

18000.000000000004

In [241]:
schedule['Current Interest Payment'].sum()

2524.7400000000002

In [244]:
def NPV(CFS:list,discount_rate:float):
    """
    CFS: list of cash flows
    discount_rate: market discount rate

    """
    npv = 0
    r = discount_rate*1e-2
    for t, cf in enumerate(CFS):
        npv += cf/((1+r)**(t))
    return np.round(npv,2)
def IRR(CFS:list):
    irr_wrapper = lambda r: NPV(CFS= CFS,discount_rate=r)
    return np.round(fsolve(irr_wrapper,.2)[0],2)

In [249]:
def PV(CF_Payment:float,discount_rate:float, T:int,freq: int):
    """
    CF_Payment: Payment Per Year
    discount_rate: current market interest rate (Example 4.21)
    T: Number of Years
    freq: Frequency of Cashflow per year
    returns present value of cash flows
    """
    CF = CF_Payment/freq
    r = (discount_rate*1e-2)/freq
    m_n = T*freq
    fac2 = r*((1+r)**(m_n))
    discount_factor = (1/r - (1/fac2))
    Present_value = CF*(discount_factor)

    return f"PV is {Present_value :,.2f}."
def PV_Bond(Notional:float,cpn:float,discount_rate:float,T:int,cpn_freq:int):
    """
    Notional: Bond principal
    cpn: coupon rate (example: 5.12)
    discount_rate : market discount rate (example 3.44)
    T: Time to maturity
    cpn_freq: times interest is paid


    """
    cpn = cpn*1e-2
    r = (discount_rate*1e-2)/cpn_freq
    CF = (Notional/cpn_freq)*(cpn)
    m_n = T*cpn_freq
    fac2 = r*((1+r)**(m_n))
    discount_factor = (1/r - (1/fac2))
    notional_discount = Notional/((1+r)**(m_n))
    final_value = CF*(discount_factor) + notional_discount
    return f"PV of bond is {final_value :,.2f}"
def morgate_payment(loan_value:float,interest:float,T:int):
    
    """
    loan_value: amount of initial loan
    interest: interest paid on the loan
    T: maturity of mortgage
    returns monthly mortgage payment


    """
    
    freq = 12
    r = (interest*1e-2)/freq
    m_n = T*freq
    #cf = (loan_value/freq)*r
    fac2 = r*((1+r)**(m_n))
    annuity = (1/r - (1/fac2))
    PMT = loan_value/annuity
    return np.round(PMT,2)
def calculate_number_periods(FV:float,Present_Value:float,cpn:float,cpn_freq:int):
    """
    FV : Future Value of Investment
    Present_Value: Present_value of Investment
    cpn: coupon interest (enter 4.22)
    cpn_freq: frequency of interest payment
    returns number of periods for life of security
    """
    cpn= (cpn*1e-2)/cpn_freq
    N = np.log(FV/Present_Value)/(np.log(1+cpn))
    return round(N)

    

    



    

In [275]:
PV_Bond(2000,5.12,5,2,2)

'PV of bond is 2,004.51'